**1. Use Pandas to repeat the Week 5 Exercise 5.7

In [22]:
# 1. Use Pandas to repeat the Week 5 Exercise 5.7
import pandas as pd
import os


# 1) generate the url for a given list of gage numbers

myGageNums = ['06803495', '06803486']
myDates = ['2000-10-10', '2019-09-30']

# generate URL
def myURL(gageList, Dates):
    gages = ('&site_no={}' * len(gageList)).format(*gageList)
    period = f'&period=&begin_date={Dates[0]}&end_date={Dates[1]}'
    return f'https://waterdata.usgs.gov/nwis/dv?&cb_00060=on&format=rdb{gages}&referred_module=sw{period}'



In [23]:
# 2) read this url and download the data to csv file


raw_df = pd.read_csv(myURL(myGageNums, myDates), skiprows=29, delimiter='\t', header=0, names=['Agency','Site','Date','Discharge_cfs','QualCode'])

raw_df.head(20)
raw_df.info()

# filter the dataframe so it includes only rows with 'USGS' data
df = raw_df[(raw_df['Agency']=='USGS')].copy()
df.head(20)

# create file path to save .csv file in current working directory
# dirPath = os.getcwd()
dirPath = r'/Users/jason/python2020summer/Kunyuanlee'
#dirPath = r'C:\Users\jason\python2020summer\Kunyuanlee'
fileName = 'USGS_Streamflow_Data.csv'
fullPath = os.path.join(dirPath, fileName)
print(fullPath)


# save fitered dataframe to .csv file
df.to_csv(fullPath, index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12315 entries, 0 to 12314
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Agency         12315 non-null  object
 1   Site           12306 non-null  object
 2   Date           12306 non-null  object
 3   Discharge_cfs  12306 non-null  object
 4   QualCode       12306 non-null  object
dtypes: object(5)
memory usage: 481.2+ KB
/Users/jason/python2020summer/Kunyuanlee/USGS_Streamflow_Data.csv


In [24]:
# 3) based on the downloaded data calculate the monthly stasticstics including
# maximum, minimum and average of the streamflow data and save it to another csv file.


# convert Date to datetime data type
df['Date']= pd.to_datetime(df['Date'])

df.info()
# change data frame index to 'Date'
# https://www.dataquest.io/blog/tutorial-time-series-analysis-with-pandas/
df = df.set_index(['Date'])

#https://datatofish.com/convert-string-to-float-dataframe/
df['Discharge_cfs'] = pd.to_numeric(df['Discharge_cfs'], errors='coerce')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12304 entries, 0 to 12314
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Agency         12304 non-null  object        
 1   Site           12304 non-null  object        
 2   Date           12304 non-null  datetime64[ns]
 3   Discharge_cfs  12304 non-null  object        
 4   QualCode       12304 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 576.8+ KB


In [25]:
# Calculate monthly statistics & write to file
# create dataframe for monthly statistics
monthlyStats = pd.DataFrame(columns=['Min','Max','Mean'])

monthlyStats.Min = df.Discharge_cfs.groupby(df['Site']).resample('M').min()
monthlyStats.Max = df.Discharge_cfs.groupby(df['Site']).resample('M').max()
monthlyStats.Mean = df.Discharge_cfs.groupby(df['Site']).resample('M').mean()


# pd.options.display.float_format = '{:.2f}'.format


monthlyStats.head(20)

# create file path to save .csv file in current working directory
# dirPath = os.getcwd()
fileName = 'monthlyStreamflowStats.csv'
fullPath = os.path.join(dirPath, fileName)

print(fullPath)

# save fitered dataframe to .csv file
monthlyStats.to_csv(fullPath, float_format='%.2f')

/Users/jason/python2020summer/Kunyuanlee/monthlyStreamflowStats.csv


In [26]:
# Calculate average annual runoff for each gage 
#convert from 'cubic feet per second' to 'acre feet per day'
df['acreFeetPerDay'] = df['Discharge_cfs']*24*60*60/43560

yearlyTotal = pd.DataFrame(columns=['acreFeet'])
yearlyTotal.acreFeet = df.acreFeetPerDay.groupby(df['Site']).resample('A').sum()
yearlyTotal

# create file path to save .csv file in current working directory
# dirPath = os.getcwd()
fileName = 'annualAcreFeet.csv'
fullPath = os.path.join(dirPath, fileName)

print(fullPath)

# save fitered dataframe to .csv file
yearlyTotal.to_csv(fullPath, float_format='%.2f')

/Users/jason/python2020summer/Kunyuanlee/annualAcreFeet.csv


2. Use Pandas to
Read data in users.zip (using Pandas' the on-the-fly decompression cabability)

Identify all the occupations and compare the user numbers between STEM-related occupations and non-STEM occupations.

Identify the locations of the users that are programmers and above 35. How many male and female programmers, respectively? Is this ratio the same for the age under 35?

Compare the numbers of male and female for each occupations

Find the occupations with the youngest and oldest mean ages, respectively

Based on the first two digits of the zip codes, find the area with the largest number of users

In [27]:
import pandas as pd
import os

# 2. Use Pandas to:
# 1. Read data in users.zip (using Pandas' the on-the-fly decompression capability)


# create file path to save .csv file in current working directory
dirPath = os.getcwd()
fileName = 'users.zip'
fullPath = os.path.join(dirPath, fileName)

# read text file from zip file
userData = pd.read_csv(fullPath, compression= 'zip', sep='|')
userData = userData.set_index('user_id')

userData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 943 entries, 1 to 943
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   age         943 non-null    int64 
 1   gender      943 non-null    object
 2   occupation  943 non-null    object
 3   zip_code    943 non-null    object
dtypes: int64(1), object(3)
memory usage: 36.8+ KB


In [28]:
 # 2. Identify all the occupations and compare the user numbers between STEM-related occupations and non-STEM occupations.

stemList = ('engineer','programmer','scientist','technician')

stemTotal = 0
for item in stemList:    
    num = len(userData[userData["occupation"] == item])
    print(f'{item} = {num}')
    stemTotal += num

totalAll = len(userData.index)
nonStemTotal = totalAll - stemTotal

print('total = ', totalAll)  
print('total STEM-related = ', stemTotal)
print('non STEM-related = ', nonStemTotal)

engineer = 67
programmer = 66
scientist = 31
technician = 27
total =  943
total STEM-related =  191
non STEM-related =  752


In [29]:
# 3. Identify the lcoations of the user that are programmers and above 35. How many male and female programmers, respecively?
#    Is this ratio the same for the age under 35?

# create dataframe for monthly statistics
programmerAgeHigh = userData.query('occupation == "programmer" & age > 35').groupby(userData['gender']).count()
maleProgrammersAgeHigh = programmerAgeHigh['gender']['M']
femaleProgrammersAgeHigh = programmerAgeHigh['gender']['F']
programmersAgeHighRatio = f'{maleProgrammersAgeHigh/femaleProgrammersAgeHigh:.2f}'

print(f" The number of male programmers over 35 = {maleProgrammersAgeHigh}")
print(f" The number of male programmers over 35 = {femaleProgrammersAgeHigh}")
print(f" The ratio of male to femail programmers over 35 = {programmersAgeHighRatio}\n")


programmerAgeLow = userData.query('occupation == "programmer" & age <= 35').groupby(userData['gender']).count()
maleProgrammersAgeLow = programmerAgeLow['gender']['M']
femaleProgrammersAgeLow = programmerAgeLow['gender']['F']
programmersAgeLowRatio = f'{maleProgrammersAgeLow/femaleProgrammersAgeLow:.2f}'

print(f" The number of male programmers 35 & under= {maleProgrammersAgeLow}")
print(f" The number of male programmers 35 & under = {femaleProgrammersAgeLow}")
print(f" The ratio of male to femail programmers 35 & under = {programmersAgeLowRatio}\n")

 The number of male programmers over 35 = 19
 The number of male programmers over 35 = 2
 The ratio of male to femail programmers over 35 = 9.50

 The number of male programmers 35 & under= 41
 The number of male programmers 35 & under = 4
 The ratio of male to femail programmers 35 & under = 10.25



In [30]:
# 4. Compare the numbers of male and female for each occupation

programmerOccupationStats = userData.groupby(['occupation','gender'])['gender'].count()
# programmerOccupationStats.rename(columns = {'age':'count'}, inplace=True)
# programmerOccupationStats = programmerOccupationStats.drop(['zip_code'], axis='columns')
print(programmerOccupationStats)

occupation     gender
administrator  F          36
               M          43
artist         F          13
               M          15
doctor         M           7
educator       F          26
               M          69
engineer       F           2
               M          65
entertainment  F           2
               M          16
executive      F           3
               M          29
healthcare     F          11
               M           5
homemaker      F           6
               M           1
lawyer         F           2
               M          10
librarian      F          29
               M          22
marketing      F          10
               M          16
none           F           4
               M           5
other          F          36
               M          69
programmer     F           6
               M          60
retired        F           1
               M          13
salesman       F           3
               M           9
scientist      F     

In [31]:

# 5. Find the occupations with the youngest and oldest 'mean' ages, respectively


meanAges = userData.groupby(['occupation'])['age'].mean()
print(meanAges.sort_values().round(decimals=2),'\n')

print(f'Answer: The {meanAges.idxmin()} is the occupation with the youngest mean age of {meanAges.min():.1f} years')
print(f'Answer: The {meanAges.idxmax()} is the occupation with the oldest mean age of {meanAges.max():.1f} years\n')

occupation
student          22.08
none             26.56
entertainment    29.22
artist           31.39
homemaker        32.57
programmer       33.12
technician       33.15
other            34.52
scientist        35.55
salesman         35.67
writer           36.31
engineer         36.39
lawyer           36.75
marketing        37.62
executive        38.72
administrator    38.75
librarian        40.00
healthcare       41.56
educator         42.01
doctor           43.57
retired          63.07
Name: age, dtype: float64 

Answer: The student is the occupation with the youngest mean age of 22.1 years
Answer: The retired is the occupation with the oldest mean age of 63.1 years



In [32]:
# 6. Based on the first two digits of the zip codes, find the area with the largest numbers of users.
userData['shortZip'] = userData['zip_code'].str[:2]
numUsers = userData.groupby(['shortZip'])['occupation'].count()
print(f" The largest number of users is in zip code prefix is '{numUsers.idxmax()}'")

 The largest number of users is in zip code prefix is '55'
